## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Quatre Cocos,MU,-20.2078,57.7625,76.08,76,37,15.66,light rain
1,Jamestown,US,42.0970,-79.2353,63.99,92,90,1.99,overcast clouds
2,Busselton,AU,-33.6500,115.3333,68.04,59,76,10.31,broken clouds
3,Shache,CN,38.4167,77.2406,53.62,15,17,3.24,few clouds
4,Santa Rosa,PH,14.3122,121.1114,90.09,63,50,11.12,scattered clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Quatre Cocos,MU,-20.2078,57.7625,76.08,76,37,15.66,light rain
8,Sur,OM,22.5667,59.5289,89.98,37,11,2.73,few clouds
10,Tiarei,PF,-17.5333,-149.3333,80.62,67,28,3.42,scattered clouds
11,Georgetown,MY,5.4112,100.3354,89.89,70,20,1.01,few clouds
12,Mahebourg,MU,-20.4081,57.7000,76.24,83,75,11.50,light rain
13,Butaritari,KI,3.0707,172.7902,81.91,74,39,10.20,scattered clouds
14,Champerico,GT,14.3000,-91.9167,81.36,84,100,8.16,overcast clouds
16,Nabire,ID,-3.3667,135.4833,86.88,66,38,5.68,light rain
21,Koungou,YT,-12.7336,45.2042,76.71,83,20,3.44,few clouds
23,Hasaki,JP,35.7333,140.8333,81.28,74,16,11.92,few clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   205
Country                205
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [19]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,76.08,light rain,-20.2078,57.7625,
8,Sur,OM,89.98,few clouds,22.5667,59.5289,
10,Tiarei,PF,80.62,scattered clouds,-17.5333,-149.3333,
11,Georgetown,MY,89.89,few clouds,5.4112,100.3354,
12,Mahebourg,MU,76.24,light rain,-20.4081,57.7000,
13,Butaritari,KI,81.91,scattered clouds,3.0707,172.7902,
14,Champerico,GT,81.36,overcast clouds,14.3000,-91.9167,
16,Nabire,ID,86.88,light rain,-3.3667,135.4833,
21,Koungou,YT,76.71,few clouds,-12.7336,45.2042,
23,Hasaki,JP,81.28,few clouds,35.7333,140.8333,


In [42]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params = params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = None
        print(f"{row['City']}:Hotel not found... skipping.")
hotel_df

Tucuma:Hotel not found... skipping.
Beloha:Hotel not found... skipping.
Gamba:Hotel not found... skipping.
Gao:Hotel not found... skipping.
Kayes:Hotel not found... skipping.
Aitape:Hotel not found... skipping.
Morros:Hotel not found... skipping.
Tangping:Hotel not found... skipping.
Yago:Hotel not found... skipping.
Lugus:Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,76.08,light rain,-20.2078,57.7625,Sunrise Attitude Hotel
8,Sur,OM,89.98,few clouds,22.5667,59.5289,Sur Plaza Hotel
10,Tiarei,PF,80.62,scattered clouds,-17.5333,-149.3333,Le Rocher de Tahiti
11,Georgetown,MY,89.89,few clouds,5.4112,100.3354,Cititel Penang
12,Mahebourg,MU,76.24,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
...,...,...,...,...,...,...,...
657,Soyo,AO,75.33,overcast clouds,-6.1349,12.3689,Hotel Mpampa
660,Ternate,ID,84.00,broken clouds,0.8000,127.4000,Grand Dafam Bela Ternate
672,El Cobre,CU,76.44,few clouds,20.0489,-75.9469,Hotel El Cobre
673,Morondava,MG,79.23,clear sky,-20.2833,44.2833,La Case Bambou


In [56]:
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             10
dtype: int64

In [57]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Quatre Cocos,MU,76.08,light rain,-20.2078,57.7625,Sunrise Attitude Hotel
8,Sur,OM,89.98,few clouds,22.5667,59.5289,Sur Plaza Hotel
10,Tiarei,PF,80.62,scattered clouds,-17.5333,-149.3333,Le Rocher de Tahiti
11,Georgetown,MY,89.89,few clouds,5.4112,100.3354,Cititel Penang
12,Mahebourg,MU,76.24,light rain,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
...,...,...,...,...,...,...,...
657,Soyo,AO,75.33,overcast clouds,-6.1349,12.3689,Hotel Mpampa
660,Ternate,ID,84.00,broken clouds,0.8000,127.4000,Grand Dafam Bela Ternate
672,El Cobre,CU,76.44,few clouds,20.0489,-75.9469,Hotel El Cobre
673,Morondava,MG,79.23,clear sky,-20.2833,44.2833,La Case Bambou


In [58]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label = "City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [74]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# 11b. Display the figure
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))